# Botnet Clustering

In [1]:
!pip install networkx pandas scikit-learn python-louvain matplotlib

  Preparing metadata (setup.py) ... done
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9389 sha256=3d0e71ee094eb044b20b795569a19611bcb7d2e8420e0651eb76a322df5c4276
  Stored in directory: /Users/sebastianschuetz/Library/Caches/pip/wheels/d0/b0/d7/6dd26c3817810fa379088eaeb755a01d9a2a411c37632079d1
Successfully built python-louvain

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [7]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import community  # python-louvain

# Load the dataset
df = pd.read_csv("cs448b_ipasn.csv")  # Ensure it has 'source', 'target', 'connections'

# Filter to only include nodes 0 to 9
filtered_df = df[(df['Source'].between(0, 9)) & (df['Target'].between(0, 9))]

# Create a weighted graph with the filtered nodes
G = nx.Graph()
for _, row in filtered_df.iterrows():
    G.add_edge(row['Source'], row['Target'], weight=row['f'])

# --- Option 1: Louvain Community Detection ---
def louvain_clustering(G):
    partition = community.best_partition(G, weight='weight')
    return partition  # Dictionary {node: community_id}

# --- Option 2: K-Means Clustering ---
def kmeans_clustering(G, num_clusters=3):  # Adjust number of clusters
    nodes = list(G.nodes())
    adj_matrix = nx.to_numpy_matrix(G, nodelist=nodes)  # Convert to adjacency matrix
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(adj_matrix)
    return dict(zip(nodes, labels))

# Choose a clustering method
use_louvain = False

if use_louvain:
    clusters = louvain_clustering(G)
else:
    clusters = kmeans_clustering(G, num_clusters=3)

# Add cluster labels to nodes
nx.set_node_attributes(G, clusters, "cluster")

# Visualize Clusters
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G, seed=42)  # Layout for visualization
colors = [clusters[node] for node in G.nodes()]
nx.draw(G, pos, node_color=colors, cmap=plt.cm.Set1, with_labels=True, node_size=800, font_size=12)
plt.title("Clustering of Nodes 0-9 Based on Communication Patterns")
plt.show()

# Save clusters to CSV
cluster_df = pd.DataFrame(list(clusters.items()), columns=['Node', 'Cluster'])
cluster_df.to_csv("node_clusters_0_to_9.csv", index=False)

AttributeError: module 'networkx' has no attribute 'to_numpy_matrix'